In [ ]:
%load_ext autoreload
%autoreload
import os
import sys
import jax
import jax.numpy as jnp
from flax import nnx
import math

%matplotlib widget
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
    
sys.path.insert(0, os.path.abspath(os.path.join('../')))
from geometric_bayesian.models import MLP
from geometric_bayesian.operators import DenseOperator, SymOperator, PSDOperator, DiagOperator
from geometric_bayesian.densities import Normal, MultivariateNormal
from geometric_bayesian.utils import DataLoader, contour_plot

### PSD Operator

In [ ]:
size = 10
rng_key = jax.random.key(0)
sca = jax.random.uniform(rng_key, (1, ))
vec = jax.random.uniform(rng_key, (size, ))
mat = jax.random.uniform(rng_key, (size, size))

In [ ]:
mat += mat.transpose() + size*jnp.eye(mat.shape[0])
op = PSDOperator(
    op=mat,
    op_type='raw'
)
op_mv = PSDOperator(
    op=lambda v : jnp.matmul(mat, v),
    op_size=size
)

In [ ]:
print("dense")
print(jnp.matmul(mat,vec).tolist())
print("op chol")
print(op(vec).tolist())
print("op mv")
print(op_mv(vec).tolist())

In [ ]:
print("dense")
print(jnp.linalg.solve(mat, vec).tolist())
print("op chol")
print(op.solve(vec).tolist())
print("op mv")
print(op_mv.solve(vec).tolist())

In [ ]:
print("dense")
print(jnp.linalg.eigh(mat)[0].tolist())
print("op chol")
print(op.diagonalize(num_iterations=10, rng_key=rng_key).tolist())
print("op mv")
print(op_mv.diagonalize(num_iterations=10, rng_key=rng_key).tolist())

### Normal

In [ ]:
mean = jnp.array(1.0)
cov = jnp.array(1.0)
pdf = Normal(mean, cov)
x = jnp.linspace(-2, 4, 100)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(x, pdf(x))

### Multivariate Normal

In [ ]:
mean = jnp.array([0.0, 0.0])
# cov = jnp.array([1.0, 0.5])
s, u = jnp.diag(jnp.array([1.0, 0.5])), jnp.array([[1.0, 1.0],[-1.0,1.0]])
cov = PSDOperator(
    op=jnp.matmul(u.transpose(), jnp.matmul(s,u)),
    op_type='raw'
)
pdf = MultivariateNormal(mean, cov)

In [ ]:
_ = contour_plot(pdf)

### GGN

In [ ]:
model = MLP(
    layers=[3, 64, 2],
    rngs=nnx.Rngs(params=0),
    prob_out=False,
)

In [ ]:
rng_key = jax.random.key(0)
train_samples = jax.random.uniform(rng_key, (10, model.shape[0]))
train_targets = jax.random.uniform(rng_key, (10, model.shape[1]))

graph_def, params = nnx.split(model)
num_params = sum(x.size for x in jax.tree.leaves(params))

In [ ]:
from geometric_bayesian.curv.ggn import ggn
ggn_mv = ggn(
    p = lambda f: MultivariateNormal(f, DiagOperator(jnp.array(1.0), model.shape[1])),
    f = model,
    X = train_samples,
    y = train_targets,
)

In [ ]:
sys.path.insert(0, os.path.abspath(os.path.join('/home/bernardo/repos/laplax/')))
from laplax.curv import create_ggn_mv
from laplax.util.loader import input_target_split

def model_fn(input,params):
    return nnx.call((graph_def, params))(input)[0]
    
ggn_mv_test = create_ggn_mv(
    model_fn,
    params,
    {
        'input': train_samples,
        'target': train_targets
    },
    loss_fn="mse",
    num_curv_samples=train_samples.shape[0],
    num_total_samples=train_samples.shape[0] // 2,
)

In [ ]:
ggn_mv(params, params)

In [ ]:
ggn_mv_test(params)

In [ ]:
from geometric_bayesian.utils.helper import wrap_pytree_function, pytree_to_array
ggn_mv = wrap_pytree_function(ggn_mv, params)
vec_params = pytree_to_array(params)
jax.vmap(ggn_mv, in_axes=(None,0))(vec_params,jnp.eye(num_params))

In [ ]:
from laplax.curv.cov import create_posterior_fn, create_full_curvature
from laplax.util.flatten import (
    create_pytree_flattener,
    wrap_factory,
    wrap_function,
)
from laplax.util.mv import diagonal, to_dense
from laplax.util.tree import get_size, eye_like

flatten, unflatten = create_pytree_flattener(params)
mv_wrapped = wrap_function(ggn_mv_test, input_fn=unflatten, output_fn=flatten)
to_dense(mv_wrapped, layout=get_size(params))

### Embedded Geometry

In [ ]:
def sphere_embedding(x, r=1.0):
    return r*jnp.array([jnp.sin(x[0])*jnp.cos(x[1]), jnp.sin(x[0])*jnp.sin(x[1]), jnp.cos(x[0])])

def surf_embedding(x):
    return jnp.array([x[0], x[1], x[0]**2 + x[1]**2])

def sphere_metric(x,r=1.0):
    return jnp.array([[1.0, 0.0],[0.0, jnp.sin(x[0])**2]])*r**2

In [ ]:
x = jnp.array([0.5,0.2])
vi = jnp.array([0.1,0.3])
vj = jnp.array([0.1,0.3]) # jnp.array([0.5,0.7])
embedding = sphere_embedding
metric = sphere_metric

In [ ]:
from geometric_bayesian.utils.math import pullback
from geometric_bayesian.operators import DiagOperator

H = DiagOperator(jnp.array(2.0), 2)
print('Reference', metric(x))
print('Identity pullback', jax.lax.map(lambda v: pullback(embedding)(x,v), jnp.eye(2)))
print('Non-identity pullback', jax.lax.map(lambda v: pullback(embedding, H)(x,v), jnp.eye(2)))

In [ ]:
from geometric_bayesian.geom.embedding import pullmetric, christoffel_fk, christoffel_sk
print('Reference', metric(x))
print('Identity pullback', jax.lax.map(lambda v: pullmetric(embedding)(x,v), jnp.eye(2)))
print('Metric-Vector product', pullmetric(embedding)(x,vi))
print('C1k-Vector product', christoffel_fk(embedding)(x,vi))
print('C2k-Vector product', christoffel_sk(embedding)(x,vi))

In [ ]:
from geometric_bayesian.utils.math import hvp, hvp_2
print('Hessian-Vector product', hvp(embedding)(x,vi))
print('Hessian-Vector product', hvp_2(embedding)(x,vi))

In [ ]:
g = pullmetric(embedding)
g_fn = lambda x: jax.lax.map(lambda v: g(x,v), jnp.eye(2))
# jax.vjp(g_fn, x)[1](jnp.vstack([vi,vi]).T)
# vi@jax.jvp(g_fn, (x,), (vi,))[1]
print("First term Christoffel", jnp.einsum('mji,i->mj', jax.jacfwd(g_fn)(x), vi)@vj)
print("Second term Christoffel", jnp.einsum('mij,i->mj', jax.jacfwd(g_fn)(x), vi)@vj)
print("Third term Christoffel", jnp.einsum('ijm,i->mj', jax.jacfwd(g_fn)(x), vi)@vj)

In [ ]:
jvp = lambda x: jax.linearize(sphere_embedding, x)[1]
vjp = lambda x: jax.linear_transpose(jvp(x), vj)
grad = lambda x: jax.lax.map(vjp(x), jnp.eye(3))
hess = lambda v: jax.jvp(grad, (x,), (v,))[1][0]

print("Hessian SC", jvp(x)(vj)@hess(vi), "Hessian DC", vjp(x)(hess(vi)@vj)[0])
print("First term Christoffel", jvp(x)(vj)@hess(vi) + vjp(x)(hess(vi)@vj)[0])

print("Hessian SC", jvp(x)(vi)@hess(vj), "Hessian DC", vjp(x)(hess(vj)@vi)[0])
print("Second term Christoffel", jvp(x)(vi)@hess(vj) + vjp(x)(hess(vj)@vi)[0])

print("Hessian SC", jvp(x)(vj)@hess(vi), "Hessian SC", jvp(x)(vi)@hess(vj))
print("Third term Christoffel", jvp(x)(vj)@hess(vi) + jvp(x)(vi)@hess(vj))

### Geodesic Integration (Embedding)

In [ ]:
import time
from functools import partial
from geometric_bayesian.geom.embedding import pullmetric, christoffel_fk, christoffel_sk

def torus(x, a=3.0, b=1.0):
    return jnp.array([(a+b*jnp.cos(x[1])) * jnp.cos(x[0]), (a+b*jnp.cos(x[1])) * jnp.sin(x[0]), b*jnp.sin(x[1])])
csk = christoffel_sk(torus)

@jax.jit
def dynamics(x,v):
    return -csk(x,v)

def ode_ef(f,x0,v0,dt,T):
    steps = int(T/dt)
    x, v = jnp.zeros((steps,len(x0))), jnp.zeros((steps, len(v0)))
    x, v = x.at[0].set(x0), v.at[0].set(v0)
    for i in range(steps-1):
        v = v.at[i+1].set(v[i] + dt*f(x[i], v[i]))
        x = x.at[i+1].set(x[i] + dt*v[i+1])
    return x, v

In [ ]:
rot = jnp.linspace(0, 2*jnp.pi, 9)[:-1]
x0 = jnp.array([[jnp.pi, jnp.pi]]).repeat(rot.shape[0],axis=0)
v0 = jnp.vstack((jnp.cos(rot),jnp.sin(rot))).transpose()
dt, T = 0.01, 2.5

start = time.time()
x,v = jax.vmap(lambda x,v: ode_ef(dynamics, x, v, dt=dt, T=T), in_axes=(0,0))(x0,v0)
print(time.time() - start)

In [ ]:
def euler_forward(f, x, v, dt):
    vn = v + dt*f(x, v)
    xn = x + dt*v
    return xn, vn

def step(carry,_):
    x,v = euler_forward(f=dynamics, x=carry[0], v=carry[1], dt=0.01)
    return (x,v), (x,v)

start = time.time()
trajectory = jax.vmap(lambda x,v: jax.lax.scan(step, (x, v), None, length=int(T/dt))[1], in_axes=(0,0))(x0,v0)
print(time.time() - start)

In [ ]:
from geometric_bayesian.utils.plot import surf_plot
fig = surf_plot(jax.vmap(torus,0), min=[0.0,0.0], max=[2*jnp.pi,2*jnp.pi], res=100)
ax = fig.axes[0]
for i in range(rot.shape[0]):
    traj = jax.lax.map(torus, trajectory[0][i])
    ax.plot(traj[:, 0], traj[:, 1], traj[:, 2], color="k")

### Geodesic Integration (Metric)

In [ ]:
import time
from geometric_bayesian.geom.embedding import pullmetric
from geometric_bayesian.geom.metric import christoffel_fk,christoffel_sk
from geometric_bayesian.geom.embedding import christoffel_sk as csk_embedding
def torus(x, a=3.0, b=1.0):
    return jnp.array([(a+b*jnp.cos(x[1])) * jnp.cos(x[0]), (a+b*jnp.cos(x[1])) * jnp.sin(x[0]), b*jnp.sin(x[1])])

g_torus = pullmetric(torus)
csk = christoffel_sk(g_torus)
csk2 = csk_embedding(torus)
cfk = christoffel_fk(g_torus)


# @jax.jit
# def dynamics(x,v):
#     return -csk(x,v)

def dynamics(t,x,u):
    x1, x2 = jnp.split(x, 2)
    return jnp.append(x2, -csk(x1,x2)), None

x_test = jnp.array([0.0000, 0.0635])

In [ ]:
g_diag = jax.vmap(lambda v: g_torus(x_test, v))(jnp.eye(2))
d,u = jnp.linalg.eigh(g_diag)
l = u*jnp.sqrt(d)

In [ ]:
g_diag

In [ ]:
jax.vmap(lambda u,v: cfk(x_test,u,v), in_axes=(1, None))(jnp.eye(2), jnp.array([0,1]))

In [ ]:
jax.vmap(lambda u,v: csk(x_test,u,v), in_axes=(1,1))(l,l).sum(axis=0)

In [ ]:
rand_3t = jax.random.uniform(jax.random.key(0), (2,2,2))
rand_2t = jax.random.uniform(jax.random.key(0), (2,2))
rand_2t = rand_2t + rand_2t.T + 10*jnp.eye(2)
d, u = jnp.linalg.eigh(rand_2t)

In [ ]:
l = u @ jnp.diag(jnp.sqrt(d))

In [ ]:
l1 = u[:,0]*jnp.sqrt(d[0])
l2 = u[:,1]*jnp.sqrt(d[1])

In [ ]:
jnp.einsum('ijk,ij->k', rand_3t, rand_2t)

In [ ]:
jnp.einsum('ijk,ij->k', rand_3t, l1[:,None] @ l1[None,:]) + jnp.einsum('ijk,ij->k', rand_3t, l2[:,None] @ l2[None,:])

In [ ]:
jax.vmap(lambda v: jnp.einsum('ijk,ij->k', rand_3t, v[:,None] @ v[None,:]), in_axes=1)(l).sum(axis=0)

In [ ]:
jnp.einsum('ijk,ij->k', rand_3t, l2[:,None] @ l2[None,:])

In [ ]:
jax.vmap(lambda u,v: jnp.einsum('jk,j->k', jnp.einsum('ijk,i->jk', rand_3t, u), v), in_axes=(1,1), out_axes=(2))(jnp.eye(2), jnp.eye(2))



In [ ]:
rand_3t

In [ ]:
jax.vmap(lambda v: csk(x_test, v))(jnp.eye(2))

In [ ]:
jax.vmap(lambda v: csk2(x_test, v))(jnp.eye(2))

In [ ]:
from geometric_bayesian.integrate.integrate import integrate
from geometric_bayesian.integrate import integrate, ef
rot = jnp.linspace(0, 2*jnp.pi, 9)[:-1]
x0 = jnp.array([[jnp.pi, jnp.pi]]).repeat(rot.shape[0],axis=0)
v0 = jnp.vstack((jnp.cos(rot),jnp.sin(rot))).transpose()
dt, T = 0.01, 2.5
step = integrate(
    f = dynamics,
    dt = dt,
    T = T,
    integrator = ef,
)

In [ ]:
start = time.time()
trajectory = jax.vmap(step, in_axes=0)(jnp.hstack((x0, v0)))
print(time.time() - start)

In [ ]:
from geometric_bayesian.utils.plot import surf_plot
fig = surf_plot(jax.vmap(torus), min=[0.0,0.0], max=[2*jnp.pi,2*jnp.pi], res=100)
ax = fig.axes[0]
for i in range(rot.shape[0]):
    traj = jax.lax.map(torus, trajectory[0][i])
    ax.plot(traj[:, 0], traj[:, 1], traj[:, 2], color="k")

In [ ]:
fig = contour_plot(jax.vmap(torus,0), min=[0.0,0.0], max=[2*jnp.pi,2*jnp.pi], res=100)

## Eigenvalues

In [ ]:
## import scipy
import numpy as np

rng_key = jax.random.key(0)
num_eig = 10
num_not_plot = 6
# eigval_scipy, eigvec_scipy = scipy.sparse.linalg.eigsh(np.array(jax.lax.map(ggn_mv, jnp.eye(num_params))), k=2)
eigval_scipy, eigvec_scipy = scipy.linalg.eigh(np.array(jax.lax.map(ggn_mv, jnp.eye(num_params))))
eigval_lanczos, eigvec_lanczos = ggn_op.diagonalize(rng_key, num_modes=num_eig, method='lanczos')
# eigval_lopcg, eigvec_lopcg = ggn_op.diagonalize(rng_key, num_modes=2, method='lobpcg')

In [ ]:
fig, ax = plt.subplots()
ax.plot(range(num_params - len(eigval_lanczos), num_params-num_not_plot), eigval_lanczos[:-num_not_plot], '-o', label='Lanczos')
ax.plot(range(num_params - len(eigval_scipy), num_params-num_not_plot), eigval_scipy[:-num_not_plot], '-o', color= 'green', label='Scipy')
ax.legend()

## Custom derivatives

In [ ]:
@jax.custom_vjp
def myfun(x):
    return x[0]**2 + 5*x[1]**3 + x[0]*x[1]
def grad_myfun(x):
    return jnp.array([2*x[0] + x[1], 15*x[1]**2 + x[0]])
def hess_myfun(x):
    return jnp.array([[2, 1],[1,30*x[1]]])

In [ ]:
def myfun_fwd(x):
  # Returns primal output and residuals to be used in backward pass by f_bwd.
  return myfun(x), ([2*x[0] + x[1], 15*x[1]**2 + x[0]])

def myfun_bwd(res, g):
  arg_1, arg_2 = res # Gets residuals computed in f_fwd
  return (jnp.array([arg_1 * g, arg_2 * g]),)

myfun.defvjp(myfun_fwd, myfun_bwd)

In [ ]:
x = jnp.array([0.2, 0.5])
v = jnp.array([0.3, 0.3])
print(grad_myfun(x))
print(jax.grad(myfun)(x))

In [ ]:
from functools import partial

p = lambda f: MultivariateNormal(f, cov=DiagOperator(1.0, model.shape[1]))

@partial(jax.custom_jvp, nondiff_argnums=(0,1))
def neg_logll(p, y, f):
    return jax.vmap(lambda y, f: -p(f)._logpdf(y), in_axes=(0,0))(y, f).sum()

def neg_logll_autograd(p, y, f):
    return jax.vmap(lambda y, f: -p(f)._logpdf(y), in_axes=(0,0))(y, f).sum()

@partial(jax.custom_jvp, nondiff_argnums=(0,1,3))
@neg_logll.defjvp
def neg_logll_jvp(p, y, primals, tangents):
    print("hello")
    f = primals[0]
    v = tangents[0]
    out = jax.vmap(lambda y, f, v: -p(f)._logpdf_jvp_mean(y, v), in_axes=(0,0,0))(y, f, v).sum(axis=0)
    return jax.ShapeDtypeStruct(out.shape, out.dtype), out

@neg_logll_jvp.defjvp
def neg_logll_hvp(p, y, dummy_tangents, primals_new, tangents_new):
    print("hello2")
    f = primals_new[0]
    v = tangents_new[0]
    out = jax.vmap(lambda y, f, v: -p(f)._logpdf_hvp_mean(y, v), in_axes=(0,0,0))(y, f, v).sum(axis=0)
    return jax.ShapeDtypeStruct(out.shape, out.dtype), out

In [ ]:
v = jax.random.uniform(rng_key, (10, model.shape[1]))

In [ ]:
neg_logll_jvp(p, train_targets, (model(train_samples),), (v,))[1]

In [ ]:
jax.jvp(lambda f: neg_logll(p, train_targets, f), (model(train_samples),), (v,))[1]

In [ ]:
jax.jvp(lambda f: neg_logll_autograd(p, train_targets, f), (model(train_samples),), (v,))[1]

In [ ]:
neg_logll_hvp(p, train_targets, None, (model(train_samples),), (v,))[1]

In [ ]:
jax.jvp(lambda f: neg_logll_jvp(p, train_targets, f, None), (model(train_samples),), (v,) )[1]

In [ ]:
jax.jvp(jax.grad(f), primals, tangents)[1]

In [ ]:
jax.jvp(jax.grad(lambda f: neg_logll(p, train_targets, f)), (model(train_samples),),(v,))[1]

In [ ]:
(lambda f: neg_logll_autograd(p, train_targets, f(train_samples)))(model)

## Posterior

In [ ]:
import jax.numpy as jnp
from jax.scipy.stats.norm import pdf as normal
from jax.scipy.stats.multivariate_normal import pdf as multivariate_normal

In [ ]:
p_theta = lambda theta: multivariate_normal(theta, mean=jnp.zeros(2), cov=jnp.array([[10,0],[0,1]]))

In [ ]:
p_x_theta = lambda x, theta: 0.5*normal(x, loc=theta[0], scale=2) + 0.5*normal(x, loc=theta.sum(), scale=2) 

In [ ]:
p_theta(jnp.array([0,0]))

In [ ]:
p_x_theta(0.1, jnp.array([0,0]))

In [ ]:
import jax
import jax.numpy as jnp
from jax import grad, random
import matplotlib.pyplot as plt

# Parameters from paper
sigma1_squared = 10
sigma2_squared = 1
sigma_x_squared = 2
num_data = 100

# True parameters
true_theta1 = 0.0
true_theta2 = 1.0

# Generate synthetic data
key = random.PRNGKey(0)
data_key, sample_key = random.split(key)
data = (random.normal(data_key, (num_data,)) * jnp.sqrt(sigma_x_squared) +
        random.choice(sample_key, jnp.array([true_theta1, true_theta1 + true_theta2]), (num_data,)))

# Log posterior function
def log_posterior(theta, data):
    theta1, theta2 = theta
    log_prior = (-0.5 * theta1**2 / sigma1_squared) + (-0.5 * theta2**2 / sigma2_squared)
    
    log_likelihood = jnp.sum(jnp.log(
        0.5 * jax.scipy.stats.norm.pdf(data, theta1, jnp.sqrt(sigma_x_squared)) +
        0.5 * jax.scipy.stats.norm.pdf(data, theta1 + theta2, jnp.sqrt(sigma_x_squared))
    ))
    
    return log_prior + log_likelihood

# Stochastic Gradient Langevin Dynamics (SGLD)
steps = 10000
gamma = 0.55
a, b = 0.01, 100
theta = jnp.array([0.0, 0.0])
samples = []

log_post_grad = grad(log_posterior)

for t in range(steps):
    eps_t = a * (b + t)**(-gamma)
    idx = t % num_data
    data_batch = data[idx:idx+1]
    stochastic_grad = log_post_grad(theta, data_batch) * num_data

    key, subkey = random.split(key)
    noise = random.normal(subkey, shape=theta.shape) * jnp.sqrt(eps_t)

    theta = theta + 0.5 * eps_t * stochastic_grad + noise
    samples.append(theta)

samples = jnp.array(samples)

# Plot samples and posterior distribution
plt.figure(figsize=(10, 6))
plt.scatter(samples[:, 0], samples[:, 1], alpha=0.2, s=1, label="SGLD Samples")
plt.xlabel(r'$\theta_1$')
plt.ylabel(r'$\theta_2$')
plt.title("Stochastic Gradient Langevin Dynamics Posterior Samples")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Grid for posterior evaluation
theta1_grid = jnp.linspace(-3, 3, 100)
theta2_grid = jnp.linspace(-3, 3, 100)
T1, T2 = jnp.meshgrid(theta1_grid, theta2_grid)
posterior_grid = jnp.exp(jax.vmap(lambda t1, t2: log_posterior(jnp.array([t1, t2]), data))(T1.ravel(), T2.ravel())).reshape(T1.shape)

# Plot samples and true posterior
plt.figure(figsize=(10, 6))
plt.contour(T1, T2, posterior_grid, levels=30, cmap='viridis')
plt.scatter(samples[:, 0], samples[:, 1], alpha=0.2, s=1, color='red', label="SGLD Samples")
plt.xlabel(r'$\theta_1$')
plt.ylabel(r'$\theta_2$')
plt.title("Stochastic Gradient Langevin Dynamics Samples with True Posterior")
plt.legend()
plt.grid(True)
plt.show()